**Update these values, and then Restart and Run All**

In [1]:
Q = 3
YEAR = 2020

In [2]:
import pandas as pd
import genetic_collections as gc
import tarfile
import io

## Downloading GenBank Records

In [3]:
q_end_dates = {1: '3/31',
               2: '6/30',
               3: '9/31',
               4: '12/31'}
bioprojects = {'GGI': '384793',
               'SIBN': '81359'}

In [4]:
project_results = []
for initiative, bioproject_code in bioprojects.items():
    gb_query = f'{bioproject_code}[BioProject] AND ("1900/01/01"[PDAT] : "{YEAR}/{q_end_dates[Q]}" [PDAT])'
    gb_search_results = gc.gb_search(raw_query=gb_query)
    print(initiative, len(gb_search_results.id_list))
    gb_results = gc.gb_fetch_from_id_list(gb_search_results.id_list)
    print(initiative, len(gb_results))
    project_df = pd.DataFrame(gb_results)
    project_df['initiative'] = initiative
    print(project_df.info())
    project_results.append(project_df)

GGI 9621
GGI 9621
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9621 entries, 0 to 9620
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   accession         9621 non-null   object
 1   scientific_name   9621 non-null   object
 2   publish_date      9621 non-null   object
 3   update_date       9621 non-null   object
 4   keyword           9621 non-null   object
 5   bioproject        9621 non-null   object
 6   seq_len           9621 non-null   object
 7   submit_authors    9621 non-null   object
 8   submit_date       9621 non-null   object
 9   submit_inst       9621 non-null   object
 10  specimen_voucher  9571 non-null   object
 11  bold_id           2092 non-null   object
 12  taxid             9621 non-null   object
 13  country           9568 non-null   object
 14  lat_lon           7718 non-null   object
 15  collection_date   7698 non-null   object
 16  collected_by      7645 non-null   object
 

In [6]:
gb_df = pd.concat(project_results)
gb_df['quarter'] = pd.to_datetime(gb_df['publish_date']).dt.year.astype(str) + '-Q' +\
                     pd.to_datetime(gb_df['publish_date']).dt.quarter.astype(str)
keep_cols = ['initiative','bioproject','accession','specimen_voucher','publish_date','quarter','taxid']
gb_df = gb_df[keep_cols]
gb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35279 entries, 0 to 25657
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   initiative        35279 non-null  object
 1   bioproject        35279 non-null  object
 2   accession         35279 non-null  object
 3   specimen_voucher  35220 non-null  object
 4   publish_date      35279 non-null  object
 5   quarter           35279 non-null  object
 6   taxid             35279 non-null  object
dtypes: object(7)
memory usage: 2.2+ MB


## Downloading NCBI Taxonomy

In [ ]:
!curl -sS -o new_taxdump.latest.tar.gz ftp://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz

In [9]:
with tarfile.open('new_taxdump.latest.tar.gz', "r:gz") as tar:
    rankedlineage = tar.extractfile('rankedlineage.dmp').read()
    ranked_taxa = pd.read_csv(io.BytesIO(rankedlineage), 
                     encoding='utf8', 
                     sep='\t\|\t', engine='python',
                     error_bad_lines=False,
                     names=['taxid','tax_name','species','genus','family',
                            'order','class','phylum','kingdom','superkingdom'])
ranked_taxa.head()

,taxid,tax_name,species,genus,family,order,class,phylum,kingdom,superkingdom
0,1,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\t|
1,131567,cellular organisms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\t|
2,2157,Archaea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\t|
3,1935183,Asgard group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea\t|
4,1936272,Candidatus Heimdallarchaeota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea\t|


In [10]:
ranked_taxa['superkingdom'] = ranked_taxa['superkingdom'].str.rstrip('\t|')
ranked_taxa.head()

,taxid,tax_name,species,genus,family,order,class,phylum,kingdom,superkingdom
0,1,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,131567,cellular organisms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,2157,Archaea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,1935183,Asgard group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea
4,1936272,Candidatus Heimdallarchaeota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea


## Combining GenBank with Taxonomy and exporting report

In [7]:
unique_taxids = gb_df['taxid'].unique()
print(len(unique_taxids))

14278


In [11]:
taxa_subset = ranked_taxa[ranked_taxa['taxid'].isin(unique_taxids)]
taxa_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14277 entries, 88303 to 2050047
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   taxid         14277 non-null  int64 
 1   tax_name      14277 non-null  object
 2   species       242 non-null    object
 3   genus         12203 non-null  object
 4   family        13875 non-null  object
 5   order         14033 non-null  object
 6   class         14041 non-null  object
 7   phylum        14277 non-null  object
 8   kingdom       14227 non-null  object
 9   superkingdom  14277 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [14]:
gb_df['taxid'] = gb_df['taxid'].astype('int')

In [15]:
merged_df = gb_df.merge(taxa_subset[['taxid','family','genus']], on='taxid')
print(merged_df.head())

  initiative   bioproject accession  specimen_voucher publish_date  quarter  \
0        GGI  PRJNA552856  MN135406  USNM:Herp:575039  17-JUL-2019  2019-Q3   
1        GGI  PRJNA552856  MN135405  USNM:Herp:575038  17-JUL-2019  2019-Q3   
2        GGI  PRJNA552856  MN135619  USNM:Herp:575039  15-JUL-2019  2019-Q3   
3        GGI  PRJNA552856  MN135618  USNM:Herp:575038  15-JUL-2019  2019-Q3   
4        GGI  PRJNA552856  MN135404  USNM:Herp:575037  17-JUL-2019  2019-Q3   

    taxid   family       genus  
0   45438  Ranidae  Lithobates  
1   45438  Ranidae  Lithobates  
2   45438  Ranidae  Lithobates  
3   45438  Ranidae  Lithobates  
4  298395  Ranidae  Lithobates  


In [17]:
merged_outfile = f'GB_Taxonomy_Results_{YEAR}_Q{Q}.tsv'
merged_df.to_csv(merged_outfile, sep='\t', index=False)

## Calculating Quarterly Totals

In [22]:
merged_df.groupby(['initiative','quarter']).size()

initiative  quarter
GGI         2016-Q1     145
            2017-Q1       8
            2017-Q2     107
            2017-Q3    2034
            2017-Q4    3040
            2018-Q1     372
            2018-Q2    1856
            2018-Q3     165
            2018-Q4    1314
            2019-Q1       2
            2019-Q2      28
            2019-Q3     400
            2019-Q4      86
            2020-Q2      61
SIBN        2011-Q4    2778
            2014-Q2      12
            2015-Q1     805
            2015-Q3     966
            2015-Q4    1836
            2016-Q1     534
            2016-Q2     498
            2016-Q4     256
            2017-Q2      27
            2017-Q4    1947
            2018-Q1     920
            2018-Q2     905
            2018-Q3    2266
            2018-Q4    1676
            2019-Q1     380
            2019-Q2     100
            2019-Q3    4726
            2019-Q4    2773
            2020-Q2    2253
dtype: int64

In [18]:
quarter_target = f'{YEAR}-Q{Q}'

sibn_q = merged_df[(merged_df['initiative'] == 'SIBN') & (merged_df['quarter'] == quarter_target)]
sibn_total = merged_df[merged_df['initiative'] == 'SIBN']
ggi_q = merged_df[(merged_df['initiative'] == 'GGI') & (merged_df['quarter'] == quarter_target)]
ggi_total = merged_df[merged_df['initiative'] == 'GGI']
both_q = merged_df[merged_df['quarter'] == quarter_target]

In [19]:
from collections import OrderedDict

In [20]:
df_labels = OrderedDict([(f'SIBN Q{Q}', sibn_q),
                         ('SIBN Total', sibn_total),
                         (f'GGI Q{Q}', ggi_q),
                         ('GGI Total', ggi_total),
                         (f'Both Initiatives Q{Q}', both_q),
                         ('Both Initiatives Total', merged_df)])

In [21]:
counts_dict = OrderedDict()
for title, df in df_labels.items():
    counts_dict[title] = OrderedDict([
                            ('Sequence Records', len(df)),
                            ('Specimens', df['specimen_voucher'].nunique()),
                            ('Families', df['family'].nunique()),
                            ('Genera', df['genus'].nunique())])
count_df = pd.DataFrame(counts_dict)
count_df.style

,SIBN Q3,SIBN Total,GGI Q3,GGI Total,Both Initiatives Q3,Both Initiatives Total
Sequence Records,0,25658,0,9618,0,35276
Specimens,0,23843,0,5236,0,29079
Families,0,926,0,763,0,1373
Genera,0,5328,0,2154,0,7181


In [24]:
summary_file = f'GenBank_Counts_{YEAR}_Q{Q}.xlsx'
count_df.to_excel(summary_file)